## Instalar pacote DEAP e importar o que for necessário

In [1]:
# https://deap.readthedocs.io/en/master/
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 5.7 MB/s eta 0:00:00


In [2]:
# Bibliotecas a serem utilizadas
from deap import algorithms, base, creator, tools
import pandas as pd
import numpy
import random

# Seed para reprodução de resultados
seed = 10
random.seed(seed)
numpy.random.seed(seed)

# Redes Neurais

In [3]:
data = pd.read_csv('BreastCancer.csv', index_col=0)
data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
data.shape

(569, 31)

In [7]:
from sklearn.model_selection import train_test_split
 
X = data.loc[:,data.columns != 'diagnosis']   # Entrada
y = data.diagnosis                            # Saída
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    stratify=y)

In [8]:
# Normalizando os Dados
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [11]:
NumberOfClasses = len(y_train.unique())

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [14]:
from keras import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical

def train_and_evaluate(hidden_layer1_size, additional_hidden_layer, additional_hidden_layer_size):
  # criar rn
  RN = Sequential()
  RN.add(Dense(hidden_layer1_size, input_shape=X_train_normalized.shape[1:], activation='sigmoid'))

  if additional_hidden_layer == 1:
    RN.add(Dense(additional_hidden_layer_size, activation='sigmoid'))

  RN.add(Dense(NumberOfClasses, activation = 'sigmoid'))
  
  # treinar rn
  RN.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy']) 
  history = RN.fit(X_train_normalized, to_categorical(y_train), epochs=100, verbose=0)

  # avaliar na base de teste
  score = RN.evaluate(X_test_normalized, to_categorical(y_test), verbose = 0)

  return score[0]

#Otimização

## Preparação para Otimização

1.   Função Objetivo
2.   Penalização
3.   Variáveis de Decisão
4.   Operadores



In [17]:
# Função Objetivo
def objective_function(individual):
  return (train_and_evaluate(individual[0],individual[1],individual[2])),

In [18]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))       # função objetivo: nome, tipo(f.o.), peso de cada objetivo (no caso só um objetivo)
creator.create("Individual", list,  fitness=creator.FitnessMin)   # indivíduo

In [19]:
def mutation(individual, indpb):

  if random.random() < indpb:
    constraints = [[5,30],[0,1], [5,30]]

    # flibit para o gene binário
    if individual[1] == 0:
      individual[1] = 1
    else:
      individual[1] = 0

    # gaussiana para número de neurônios
    for idx, gene in enumerate(individual):
      if idx != 1:  # o ninário não vai entrar aqui
        rnd = random.gauss(0, 2)
        new_gene = int(round(gene + rnd))

        while not constraints[idx][0] <= new_gene <= constraints[idx][1]:
          rnd = random.gauss(0, 1)
          new_gene = int(round(gene + rnd))

        individual[idx] = new_gene
  return individual,

In [20]:
def layer_size():
  return random.randint(5, 30)

def additional_hidden_layer():
  return random.randint(0, 1)

In [21]:
toolbox = base.Toolbox()

# Inicializador de indivíduo e população
toolbox.register("individual", tools.initCycle, creator.Individual, [layer_size, additional_hidden_layer, layer_size], 1)  
toolbox.register("population", tools.initRepeat, list, toolbox.individual)                  # lista de indivíduos, n vai ser passado embaixo quando eu chamar toolbox.population

In [22]:
# Incializador de operadores
toolbox.register("evaluate", objective_function)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", mutation, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [23]:
pop = toolbox.population(n=20)                            # inicialização da pop
hof = tools.HallOfFame(1)                                # melhor indivíduo
stats = tools.Statistics(lambda ind: ind.fitness.values)  # estatísticas
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

## Otimização

In [ ]:
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.1, ngen=30, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg     	std     	min     	max     
0  	20    	0.361886	0.151667	0.173101	0.549516
1  	11    	0.23973 	0.0951869	0.175027	0.489641
2  	12    	0.189757	0.0124204	0.17116 	0.217447
3  	13    	0.186934	0.014492 	0.167557	0.216021
4  	16    	0.191885	0.0170595	0.164556	0.240782
5  	9     	0.184075	0.0151919	0.164556	0.236654
6  	11    	0.187417	0.016244 	0.172393	0.242252
7  	7     	0.181383	0.00876327	0.16896 	0.201973
8  	13    	0.185377	0.0120039 	0.16896 	0.206134
9  	11    	0.178513	0.0099852 	0.16896 	0.203147
10 	12    	0.179712	0.0122491 	0.167884	0.207248
11 	9     	0.175853	0.00956689	0.162191	0.196776
12 	12    	0.18148 	0.0135671 	0.162191	0.217213
13 	7     	0.175002	0.0104083 	0.162191	0.193731
14 	11    	0.179745	0.0120093 	0.162191	0.202437
15 	14    	0.18361 	0.0156669 	0.162191	0.220929


## Resultados

In [ ]:
# Melhor solução
print("Melhor Indivíduo:")
print(hof[0])

# Melhor resultado da função objetivo
print("Melhor Resultado da Função Objetivo:")
objective_function(hof[0])

Retornar o modelo para fazer inferências e calcular acurácia